In [66]:
using ModelingToolkit, IntervalSets, Sophon, CairoMakie, Lux
using Optimization, OptimizationOptimJL

@parameters x,t
@variables θ(..), w(..)
Dₜ = Differential(t)
Dₜ² = Differential(t)^2
Dₓ = Differential(x)
Dₓ² = Differential(x)^2

g(x,t) = cos(t) - π/2 * sin(x)*cos(t)

eqs = [
    Dₜ²(θ(x,t)) - Dₓ²(θ(x,t)) + θ(x,t) - Dₓ(w(x,t)) ~ 0.0,
    Dₜ²(w(x,t)) - Dₓ²(w(x,t)) + Dₓ(θ(x,t)) ~ g(x,t)]

bcs = [
    θ(x, 0.0) ~ π/2*cos(x) + x - π/2,
    Dₜ(θ(x, 0.0)) ~ 0.0,
    w(x, 0.0) ~ π/2*sin(x),
    Dₜ(w(x, 0.0)) ~ 0.0,
    θ(0.0, t) ~ 0.0,
    θ(π, t) ~ 0.0,
    w(0.0, t) ~ 0.0, 
    w(π, t) ~ 0.0]

    
domains = [x ∈ Interval(0.0, π),
           t ∈ Interval(0.0, 1.0)]

@named timo = PDESystem(eqs, bcs, domains, [x,t], [θ(x,t),w(x,t)])

PDESystem
Equations: Equation[Differential(t)(Differential(t)(θ(x, t))) + θ(x, t) - Differential(x)(Differential(x)(θ(x, t))) - Differential(x)(w(x, t)) ~ 0.0, Differential(t)(Differential(t)(w(x, t))) + Differential(x)(θ(x, t)) - Differential(x)(Differential(x)(w(x, t))) ~ cos(t) - 1.5707963267948966cos(t)*sin(x)]
Boundary Conditions: Equation[θ(x, 0.0) ~ x + 1.5707963267948966cos(x) - 1.5707963267948966, Differential(t)(θ(x, 0.0)) ~ 0.0, w(x, 0.0) ~ 1.5707963267948966sin(x), Differential(t)(w(x, 0.0)) ~ 0.0, θ(0.0, t) ~ 0.0, θ(π, t) ~ 0.0, w(0.0, t) ~ 0.0, w(π, t) ~ 0.0]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(x, 0.0..3.141592653589793), Symbolics.VarDomainPairing(t, 0.0..1.0)]
Dependent Variables: Num[θ(x, t), w(x, t)]
Independent Variables: Num[x, t]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

In [67]:
pinn = PINN(θ = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3),
            w = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3))

sampler = QuasiRandomSampler(2000, 500)
strategy = NonAdaptiveTraining(1, 10)

prob = Sophon.discretize(timo, pinn, sampler, strategy)

OptimizationProblem. In-place: true
u0: ComponentVector{Float64}(θ = (layer_1 = (weight = [-0.849162757396698 -1.526961326599121; 0.7668944001197815 0.4194084405899048; … ; -0.840292751789093 -0.8789089918136597; 1.2842298746109009 0.2304474264383316], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = [-0.2226192206144333 0.15979890525341034 … 0.45710548758506775 0.6355644464492798; 0.12449562549591064 -0.7194952964782715 … -0.022265568375587463 -0.18508882820606232; … ; 0.30596956610679626 -0.10392359644174576 … 0.15384256839752197 0.462260365486145; 0.09861060976982117 0.6622305512428284 … -0.6874988675117493 0.5186225771903992], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = [0.2242422103881836 0.2549292743206024 … -0.010633368976414204 -0.2915177643299103; 0.08219314366579056 0.1889810860157013 … -0.23082846403121948 -0.1989176869392395; … ; -0.4000178277492523 -0.43325185775756836 … 0.25358355045318604 0.3031925559043884; -0.05106069892644882 0.12508821487426758 … 0

In [68]:
function callback(p, l)
    println("Loss: $l")
    return false
end

@time res = Optimization.solve(prob, BFGS(); maxiters=3000, callback=callback)

Loss: 186.9057652819791


Loss: 36.05873955127028


Loss: 23.768648509023183


Loss: 18.125140865423674


Loss: 16.290076224852438


...

Loss: 2.988451999626072e-5


Loss: 2.9526657902693683e-5


Loss: 2.9177216983534756e-5


Loss: 2.8669138998004526e-5


Loss: 2.8325502180847115e-5


In [ ]:
phi = pinn.phi
ps = res.u

xs, ts= [infimum(d.domain):0.01:supremum(d.domain) for d in timo.domain]

θ = [sum(phi.θ(([x,t]), ps.θ)) for x in xs, t in ts]
w = [sum(phi.w(([x,t]), ps.w)) for x in xs, t in ts]

fig = Figure(resolution=(1200,500))
axis = (xlabel="t", ylabel="x", title="θ")
heatmap(fig[1,1], xs, ts, θ, axis=axis, colormap=:jet)
heatmap(fig[1, 2], xs, ts, w, axis= merge(axis, (; title="w")), colormap=:jet)
display(fig)

In [ ]:
θ_analytic(x,t) = (π/2*cos(x) + x - π/2)*cos(t)
w_analytic(x,t) = π/2*sin(x)*cos(t)

θ_true = [θ_analytic(x,t) for x in xs, t in ts]
w_true = [w_analytic(x,t) for x in xs, t in ts]

axis = (xlabel="t", ylabel="x", title="abs(θ - θ_true)")
fig, ax1, hm1 = heatmap(xs, ts, abs.(θ .- θ_true), axis=axis, colormap=:jet)
ax2, hm2= heatmap(fig[1, 2], xs, ts, abs.(w .- w_true), axis= merge(axis, (; title="abs(w - w_true)")), colormap=:jet)
display(fig)